In [ ]:
!pip install rich-theme-manager
from rich.console import Console
from rich.style import Style
import pathlib
from rich_theme_manager import Theme, ThemeManager

THEMES = [
    Theme(
        name="dark",
        description="Dark mode theme",
        tags=["dark"],
        styles={
            "repr.own": Style(color="#e87d3e", bold=True),
            "repr.tag_name": "dim cyan",
            "repr.call": "bright_yellow",
            "repr.str": "bright_green",
            "repr.number": "bright_red",
            "repr.none": "dim white",
            "repr.attrib_name": Style(color="#e87d3e", bold=True),
            "repr.attrib_value": "bright_blue",
            "default": "bright_white on black"
        },
    ),
    Theme(
        name="light",
        description="Light mode theme",
        styles={
            "repr.own": Style(color="#22863a", bold=True),          #Class names
            "repr.tag_name": Style(color="#00bfff", bold=True),     #Adjust tag names
            "repr.call": Style(color="#ffff00", bold=True),         #Function calls and other symbols
            "repr.str": Style(color="#008080", bold=True),          #String representation
            "repr.number": Style(color="#ff6347", bold=True),       #Numbers
            "repr.none": Style(color="#808080", bold=True),         #None
            "repr.attrib_name": Style(color="#ffff00", bold=True),  #Attribute names
            "repr.attrib_value": Style(color="#008080", bold=True), #Attribute values
            "default": Style(color="#000000", bgcolor="#ffffff"),   #Default text and background
        },
    ),
]

theme_dir = pathlib.Path("themes").expanduser()
theme_dir.expanduser().mkdir(parents=True, exist_ok=True)

theme_manager = ThemeManager(theme_dir=theme_dir, themes=THEMES)
theme_manager.list_themes()

dark = theme_manager.get("dark")
theme_manager.preview_theme(dark)

 Theme  Description       Tags  Path               
 dark   Dark mode theme   dark  themes/dark.theme  
 light  Light mode theme        themes/light.theme

                                      Theme: dark - themes/dark.theme                                      
┌───────────────────┬───────────────┬───────┬─────────┬─────────┬────────────────┬────────────────────────┐
│ style             │ color         │ color │ bgcolor │ bgcolor │ attributes     │ example                │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.own          │ #e87d3e       │ █████ │ None    │         │ b------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.tag_name     │ cyan          │ █████ │ None    │         │ -d------------ │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.call         │ bright_yellow │ █████ │ None    │         │ -------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.str          │ bright_green  │ █████ │ None    │         │ -------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.number       │ bright_red    │ █████ │ None    │         │ -------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.none         │ white         │ █████ │ None    │         │ -d------------ │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.attrib_name  │ #e87d3e       │ █████ │ None    │         │ b------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.attrib_value │ bright_blue   │ █████ │ None    │         │ -------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ default           │ bright_white  │ █████ │ black   │ █████   │ -------------- │ The quick brown fox... │
└───────────────────┴───────────────┴───────┴─────────┴─────────┴────────────────┴────────────────────────┘
┌─ attributes legend ──────────────────────────────────────────────────────────────────┐
│  b: bold, d: dim, i: italic, u: underline, U: double underline, B: blink, 2: blink2  │
│  r: reverse, c: conceal, s: strike, f: frame, e: encircle, o: overline, L: Link      │
└──────────────────────────────────────────────────────────────────────────────────────┘

In [ ]:
from rich.console import Console

dark = theme_manager.get("dark")
#Console with the dark theme
console = Console(theme=dark)

In [ ]:
import warnings

#Warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd

data = (
    pd.read_csv(
        '/content/sample_data/microbiologyevents.csv',
        on_bad_lines='skip',
        engine='python'
    )
    .reset_index(drop=True)
    .to_dict('records')
)
console.print(data[:2])

[
    {
        'microevent_id': 1914554,
        'subject_id': 14797383,
        'micro_specimen_id': 707787,
        'order_provider_id': 'Not specified',
        'spec_itemid': 70012,
        'spec_type_desc': 'BLOOD CULTURE',
        'test_seq': 1,
        'test_itemid': 90201,
        'test_name': 'Blood Culture, Routine',
        'comments': 'NO GROWTH.  '
    },
    {
        'microevent_id': 2665670,
        'subject_id': 16703369,
        'micro_specimen_id': 3610427,
        'order_provider_id': 'Not specified',
        'spec_itemid': 70079,
        'spec_type_desc': 'URINE',
        'test_seq': 1,
        'test_itemid': 90039,
        'test_name': 'URINE CULTURE',
        'comments': '<10,000 organisms/ml.  '
    }
]

In [ ]:
!pip install qdrant-client sentence-transformers
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

#Creating the vector database client
qdrant = QdrantClient(":memory:")

#Creating the embedding encoder
encoder = SentenceTransformer('pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb') #Model to create embeddings

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.5/378.5 kB 4.0 MB/s eta 0:00:00


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#Creating collection to store the medical report data
collection_name="Medical_reports"

qdrant.recreate_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),
        distance=models.Distance.COSINE
    )
)

True

In [ ]:
#Vectorize!
qdrant.upload_points(
    collection_name=collection_name,
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["comments"]).tolist(),
            payload=doc
        ) for idx, doc in enumerate(data)
    ]
)

In [ ]:
console.print(qdrant.get_collection(collection_name=collection_name))

CollectionInfo(
    status=<CollectionStatus.GREEN: 'green'>,
    optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>,
    warnings=None,
    indexed_vectors_count=0,
    points_count=39882,
    segments_count=1,
    config=CollectionConfig(
        params=CollectionParams(
            vectors=VectorParams(
                size=768,
                distance=<Distance.COSINE: 'Cosine'>,
                hnsw_config=None,
                quantization_config=None,
                on_disk=None,
                datatype=None,
                multivector_config=None
            ),
            shard_number=None,
            sharding_method=None,
            replication_factor=None,
            write_consistency_factor=None,
            read_fan_out_factor=None,
            on_disk_payload=None,
            sparse_vectors=None
        ),
        hnsw_config=HnswConfig(
            m=16,
            ef_construct=100,
            full_scan_threshold=10000,
            max_indexing_threads=0,
            on_disk=None,
            payload_m=None,
            inline_storage=None
        ),
        optimizer_config=OptimizersConfig(
            deleted_threshold=0.2,
            vacuum_min_vector_number=1000,
            default_segment_number=0,
            max_segment_size=None,
            memmap_threshold=None,
            indexing_threshold=20000,
            flush_interval_sec=5,
            max_optimization_threads=1
        ),
        wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0, wal_retain_closed=1),
        quantization_config=None,
        strict_mode_config=None,
        metadata=None
    ),
    payload_schema={}
)

In [ ]:
user_prompt = "Give me the details of gram stain results that found polymorphonuclear leukocytes but no microorganisms"

In [ ]:
query_vector = encoder.encode(user_prompt).tolist()

In [ ]:
# hits = qdrant.query_points(
#     collection_name=collection_name,
#     query=query_vector,
#     limit=5
# )

In [ ]:
from rich.console import Console
from rich.text import Text
from rich.table import Table

console = Console()

table = Table(title="Retrieval Results", show_lines=True)

#Medical data columns
table.add_column("Subject ID", style="#e0e0e0")
table.add_column("Test Name", style="bright_red")
table.add_column("Comments", style="#89ddff")
table.add_column("Score", style="#a6accd")

results = qdrant.query_points(
    collection_name=collection_name,
    query=query_vector,
    limit=3
)

for hit in results.points:
    table.add_row(
        str(hit.payload.get("subject_id", "N/A")),
        hit.payload.get("test_name", "N/A"),
        f'{hit.payload.get("comments", "")[:100]}...',
        f"{hit.score:.4f}"
    )

console.print(table)

                                                 Retrieval Results                                                 
┏━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Subject ID ┃ Test Name  ┃ Comments                                                                     ┃ Score  ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 15338322   │ GRAM STAIN │ NO POLYMORPHONUCLEAR LEUKOCYTES SEEN.  NO MICROORGANISMS SEEN.  SMEAR        │ 0.7833 │
│            │            │ REVIEWED; RESULTS CONFIRMED.  ...                                            │        │
├────────────┼────────────┼──────────────────────────────────────────────────────────────────────────────┼────────┤
│ 18964292   │ GRAM STAIN │ NO POLYMORPHONUCLEAR LEUKOCYTES SEEN.  NO MICROORGANISMS SEEN.  SMEAR        │ 0.7833 │
│            │            │ REVIEWED; RESULTS CONFIRMED.  ...                                            │        │
├────────────┼────────────┼──────────────────────────────────────────────────────────────────────────────┼────────┤
│ 18109577   │ GRAM STAIN │ NO POLYMORPHONUCLEAR LEUKOCYTES SEEN.  NO MICROORGANISMS SEEN.  SMEAR        │ 0.7833 │
│            │            │ REVIEWED; RESULTS CONFIRMED.  ...                                            │        │
└────────────┴────────────┴──────────────────────────────────────────────────────────────────────────────┴────────┘

In [ ]:
from dotenv import load_dotenv

load_dotenv()

False

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from rich.panel import Panel
from rich.text import Text
from rich.console import Console

console = Console()

print("Loading TinyLlama...")
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
model = AutoModelForCausalLM.from_pretrained(
    "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    torch_dtype=torch.float16,
    device_map="auto"
)
print("Model loaded!")

#TinyLlama format
system_message = "You are a medical assistant. Your priority is to help analyze medical test results and provide clear, accurate information."
prompt = f"<|system|>\n{system_message}</s>\n<|user|>\n{user_prompt}</s>\n<|assistant|>\n"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=300, temperature=0.7)

response_text_str = tokenizer.decode(outputs[0], skip_special_tokens=True)
response_text_str = response_text_str.split("<|assistant|>")[-1].strip()

styled_panel = Panel(
    Text(response_text_str),
    title="Report Analysis without Retrieval",
    expand=False,
    border_style="bold green",
    padding=(1, 1)
)

console.print(styled_panel)

Loading TinyLlama...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Model loaded!


╭─────────────────────────────────────── Report Analysis without Retrieval ───────────────────────────────────────╮
│                                                                                                                 │
│ Gram stain results are a type of microbiological test used to identify and differentiate bacteria, fungi, and   │
│ other microorganisms. In the context of medical practice, polymorphonuclear leukocytes (PNLs) are white blood   │
│ cells that are found in the blood and lymphatic system. PNLs are typically found in the bloodstream and are     │
│ involved in the immune response.                                                                                │
│                                                                                                                 │
│ In the context of medical practice, a gram stain is a microscopic examination of a sample of tissue or fluid to │
│ identify the presence of bacteria, fungi, and other microorganisms. In the context of medical practice, a gram  │
│ stain result that found PNLs but no microorganisms is a positive result. This means that the sample contained   │
│ PNLs, but there were no microorganisms present.                                                                 │
│                                                                                                                 │
│ In summary, a gram stain result that found PNLs but no microorganisms is a positive result, indicating that the │
│ sample contains PNLs and does not contain any microorganisms.                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
#Variable to hold the search results
search_results = [hit.payload for hit in results.points]

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from rich.panel import Panel
from rich.text import Text
from rich.console import Console

console = Console()

print("Loading TinyLlama...")
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
model = AutoModelForCausalLM.from_pretrained(
    "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    torch_dtype=torch.float16,
    device_map="auto"
)
print("Model loaded!")

#TinyLlama format
system_message = "You are a medical assistant. Your priority is to help analyze medical test results and provide clear, accurate information."
prompt = f"<|system|>\n{system_message}</s>\n<|user|>\n{user_prompt}</s>\n<|results|>\n{search_results}</s>\n"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=300, temperature=0.7)

response_text_str = tokenizer.decode(outputs[0], skip_special_tokens=True)
response_text_str = response_text_str.split("<|assistant|>")[-1].strip()

styled_panel = Panel(
    Text(response_text_str),
    title="Report Analysis with Retrieval",
    expand=False,
    border_style="bold green",
    padding=(1, 1)
)

console.print(styled_panel)

Loading TinyLlama...
Model loaded!


╭──────────────────────────────────────── Report Analysis with Retrieval ─────────────────────────────────────────╮
│                                                                                                                 │
│ Gram stain results found polymorphonuclear leukocytes but no microorganisms. The test sequence was 1, test      │
│ itemid was 90270, and the test name was "GRAM STAIN." The microevent_id for the test result was 2131095. The    │
│ subject_id was 15338322, the order_provider_id was "P29U87," the spec_itemid was 70003, the spec_type_desc was  │
│ "ABSCESS," and the test_seq was 1. The test_itemid was 90270, and the test_name was "GRAM STAIN." The comments  │
│ for the test result were "NO POLYMORPHONUCLEAR LEUKOCYTES SEEN.  NO MICROORGANISMS SEEN.  SMEAR REVIEWED;       │
│ RESULTS CONFIRMED."                                                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯